In [2]:
%load_ext autoreload
%autoreload 2

import os
import time

import sys
sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from joblib import Parallel, delayed

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.filters.rank import entropy
from skimage.morphology import remove_small_objects, disk
from skimage.measure import label, regionprops
from itertools import product

In [3]:
section_ind = 66
dm = DataManager(stack='MD589', section=section_ind, segm_params_id='tSLIC200', load_mask=True)
# dm.mask = rescale(dm.thumbmail_mask.astype(np.bool), 32).astype(np.bool)
# display(dm.mask)

In [6]:
dm.ymax

7663

In [2]:
def generate_mask(img):
    
#     img = denoise_bilateral(img, win_size=5)

    h, w = img.shape
    
#     v = np.zeros_like(img, np.bool)
#     for r, c in product(range(h), range(w)):
#             nbr = img[max(0,r-5):min(h, r+5), max(0, c-5):min(w, c+5)]
#             v[r,c] = np.std(nbr) > 0.005 and 1. not in nbr

    e = entropy(img, disk(5))
    v = e > 3

    l = label(v, background=0)
    mask = l == np.argmax([p.area for p in regionprops(l+1)])
    
    mask = ~remove_small_objects(~mask, min_size=10000, connectivity=8)
    
    l = label(v)
    l[v > 0] = -1
    props = regionprops(l)
    
    border_holes = np.where([np.any(p.coords[:,0] == 0) or np.any(p.coords[:,1] == 0) \
                             or np.any(p.coords[:,0] == h-1) or np.any(p.coords[:,1] == w-1) 
                             for p in props])[0]

    for i in border_holes:
        c = props[i].coords
        mask[c[:,0], c[:,1]] = 0
    
    return mask

In [10]:
def f(stack, sec):
    img = rgb2gray(imread('/home/yuncong/CSHL_data_processed/'+stack+'_thumbnail_aligned_cropped/'+stack+'_%04d_thumbnail_aligned_cropped.tif'%sec))
    mask = generate_mask(img)
    
    img2 = img.copy()
    img2[~mask] = 0

    imsave('/home/yuncong/CSHL_data_processed/'+stack+'_thumbnail_aligned_cropped_mask/'+stack+'_%04d_thumbnail_aligned_cropped_mask.png'%sec, img_as_float(mask))
    imsave('/home/yuncong/CSHL_data_processed/'+stack+'_thumbnail_aligned_cropped_masked/'+stack+'_%04d_thumbnail_aligned_cropped_masked.png'%sec, img2)

In [4]:
section_range_lookup = {'MD593': (41,176), 'MD594': (47,186), 'MD595': (35,164), 'MD592': (46,185), 'MD589':(49,186)}

In [11]:
stack ='MD589'

mask_dir = '/home/yuncong/CSHL_data_processed/'+stack+'_thumbnail_aligned_cropped_mask/'
masked_tb_dir = '/home/yuncong/CSHL_data_processed/'+stack+'_thumbnail_aligned_cropped_masked/'

if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)
if not os.path.exists(masked_tb_dir):
    os.makedirs(masked_tb_dir)

first, last = section_range_lookup[stack]
    
_ = Parallel(n_jobs=16)(delayed(f)(stack, sec) for sec in range(first, last+1))

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/external/tifffile/tifffile_local.py:3246: UserWarning: unexpected end of lzw stream (code 514)
  warnings.warn("unexpected end of lzw stream (code %i)" % code)
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/external/tifffile/tifffile_local.py:3246: UserWarning: unexpected end of lzw stream (code 514)
  warnings.warn("unexpected end of lzw stream (code %i)" % code)
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/util/dtype.py:111: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/util/dtype.py:111: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/oasis/projects/nsf/

In [ ]:
img = rgb2gray(imread('/home/yuncong/CSHL_data_processed/MD595_thumbnail_aligned_cropped/MD595_0115_thumbnail_aligned_cropped.tif'))

In [ ]:
# img = denoise_bilateral(auto, win_size=5)

h, w = img.shape
e = entropy(img, disk(5))
v = e > 3

plt.figure();
plt.matshow(e);
plt.colorbar();

plt.figure();
plt.matshow(v, cmap=plt.cm.gray);

# v = np.zeros_like(img, np.bool)
# v = np.zeros_like(img)
# for r, c in product(range(h), range(w)):
#     nbr = img[max(0,r-5):min(h, r+5), max(0, c-5):min(w, c+5)]
# #     v[r,c] = np.std(nbr)
#     m = np.mean(nbr)
#     v[r,c] = np.mean((nbr-m)**4)/np.mean((nbr-m)**2)**2
#     v[r,c] = np.std(nbr) > 0.005 and 1. not in nbr
#     v[r,c] = np.std(nbr) > 0.15 and 1. not in nbr

l = label(v, background=0)
mask = l == np.argmax([p.area for p in regionprops(l+1)])

plt.figure();
plt.matshow(v, cmap=plt.cm.gray);

mask = ~remove_small_objects(~mask, min_size=10000, connectivity=8)

l = label(v)
l[v > 0] = -1
props = regionprops(l)

border_holes = np.where([np.any(p.coords[:,0] == 0) or np.any(p.coords[:,1] == 0) \
                         or np.any(p.coords[:,0] == h-1) or np.any(p.coords[:,1] == w-1) 
                         for p in props])[0]

for i in border_holes:
    c = props[i].coords
    mask[c[:,0], c[:,1]] = 0

img2 = img.copy()
img2[~mask] = 0
# plt.imshow(img2, cmap=plt.cm.gray);

display(img2)

# imsave('/home/yuncong/CSHL_data_processed/MD593_thumbnail_mask/MD593_%04d_thumbnail_aligned_cropped_mask.png'%171, img_as_float(mask))